In [77]:
import pandas as pd
from extract.get_data_from_kaggle import download_crime_dataset, download_mental_health_dataset
from transform.transform_functions import transform_country_names

### Extract data from source (Kaggle)

In [78]:
crime_dataset_path = download_crime_dataset() + "/crime-trends-and-operations-of-criminal-justice-systems-un-cts-csv-1.csv"
mental_health_dataset_path = download_mental_health_dataset() + "/Mental health Depression disorder Data.csv"

In [79]:
# Load csv files into pandas dataframes
crime_df = pd.read_csv(crime_dataset_path)
mental_health_df = pd.read_csv(mental_health_dataset_path, index_col=0, low_memory=False)

### Transform

Remove second dataset that is a part of the mental health disorders one


In [80]:
new_dataset_index = mental_health_df[mental_health_df['Entity'] == 'Entity'].index[0]
mental_health_df = mental_health_df.loc[:new_dataset_index - 1]

Remove unneeded columns, and coerce other columns to ensure numeric

In [81]:
mhd_columns_to_remove = ["Code", "Bipolar disorder (%)", "Eating disorders (%)"]
mental_health_df = mental_health_df.drop(columns=mhd_columns_to_remove)

crime_df = crime_df.drop(columns=["sexual violence"])

mhd_numeric_columns = ["Year", "Schizophrenia (%)", "Anxiety disorders (%)", "Drug use disorders (%)", "Depression (%)", "Alcohol use disorders (%)"]
crime_numeric_columns = ["date", "rate"]

# Convert mental health columns to numeric
for col in mhd_numeric_columns:
    mental_health_df[col] = pd.to_numeric(mental_health_df[col], errors="coerce")


# Convert crime to numeric
for col in crime_numeric_columns:
    crime_df[col] = pd.to_numeric(crime_df[col], errors="coerce")

Rename crime's columns to ensure they match mental health dataset

In [82]:
crime_df.rename(columns={"country/territory": "Entity", "date":"Year", "rate": "Sexual Violence Rate"}, inplace=True)

Update the names of the territories to ensure both datasets share the same country names


In [83]:
crime_df = crime_df.apply(transform_country_names, axis=1)

Concatenate the 'England' and 'Wales' rows into a new one

In [84]:
# Get averaged values for each year
filtered = mental_health_df[mental_health_df['Entity'].isin(['England', 'Wales'])]
filtered = filtered.groupby('Year').mean(numeric_only=True).reset_index()
# Drop original England and Wales rows
mental_health_df = mental_health_df[~mental_health_df['Entity'].isin(['England', 'Wales'])]

In [85]:
# Append the filtered DF to the mental health one 
filtered["Entity"] = "United Kingdom (England and Wales)"
mental_health_df = pd.concat([mental_health_df, filtered]).reset_index(drop=True)
mental_health_df

,Entity,Year,Schizophrenia (%),Anxiety disorders (%),Drug use disorders (%),Depression (%),Alcohol use disorders (%)
0,Afghanistan,1990,0.160560,4.828830,1.677082,4.071831,0.672404
1,Afghanistan,1991,0.160312,4.829740,1.684746,4.079531,0.671768
2,Afghanistan,1992,0.160135,4.831108,1.694334,4.088358,0.670644
3,Afghanistan,1993,0.160037,4.830864,1.705320,4.096190,0.669738
4,Afghanistan,1994,0.160022,4.829423,1.716069,4.099582,0.669260
...,...,...,...,...,...,...,...
6435,United Kingdom (England and Wales),2013,0.254036,4.652097,1.653962,4.081280,1.738443
6436,United Kingdom (England and Wales),2014,0.254957,4.652467,1.644664,4.083395,1.740366
6437,United Kingdom (England and Wales),2015,0.256066,4.653243,1.634713,4.085584,1.740498
6438,United Kingdom (England and Wales),2016,0.257352,4.654402,1.623996,4.089443,1.738599


Remove 

In [86]:
crime_df = crime_df.dropna()
res = pd.merge(mental_health_df, crime_df, on=["Entity", "Year"])